# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Необходимо построить модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Откройте файл с данными и изучите его. 
Путь к файлу: /datasets/users_behavior.csv. Импортируем все потенциально необходимые библиотеки и классификаторы.

In [19]:
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
try:
    df = pd.read_csv("/datasets/users_behavior.csv") 

except:
    print("Есть ошибки")
else:
    print("Все загрузилось!")

In [3]:
df.info()

In [4]:
df.shape

Выведем на экран наименования столбцов

In [5]:
df.columns

In [6]:
display(df.head(10))

В таблице  5 столбцов и 3214 строк. 
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
- Типы данных соответствуют, пропусков нет.

## Разбейте данные на выборки

Разделите исходные данные на обучающую, валидационную и тестовую выборки.

В нашем случае нет отдельно выделенной тестовой выборки, поэтому существющие данные мы делим на три выборки:

- обучающая (для обучения модели)
- валидационная (для выбора лучшего алгоритма и оптимальных параметров)
- тестовая (для финального теста лучшей модели).

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [20]:
#разбиваем на 3 выборки:обучающую, валидационную, тестовую
features_train, features_part, target_train, target_part = train_test_split(features, target, test_size=0.30, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_part, target_part, test_size=0.5, random_state=12345)

In [10]:
print(features_train.shape, features_valid.shape, features_test.shape, target_train.shape,  target_valid.shape, target_test.shape)

Мы разделили тестовую, тренировочную выборку и валидационную на признаки(features) и целевой признак(target).

## Исследуйте модели

Обучим модель "Дерево Решений" и проверим разные глубины

In [13]:
dd = dict() #словарь (глубина, погрешность)
for depth in range(1, 6): 
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    dd[depth] = accuracy
    print('depth:', depth, 'accuracy:', accuracy)
print('лучшая модель глубиной', [key for key in dd if dd[key] == max(dd.values())])

Обучим модель алгоритм "Случайный лес" + различная глубина

In [14]:
accur=0
est=0
dep=0
for estim in range(2,15, 3): 
    for depth in range(11,15):
        model = RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        if accuracy > accur:
            accur=accuracy
            est=estim
            dep=depth
print("n_estimators:", est, "depth:", dep, "accuracy", accur)

Обучим модель алгоритмом "логистической регрессии"

In [15]:
model =  LogisticRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('accuracy:', accuracy)

Обучили "дерево принятия решений".
Обучили "модель случайного леса" с подбором глубины. 
Обучили "модель логистической регрессии" и получили метрики хуже остальных моделей.

## Проверьте модель на тестовой выборке

Объединение тренировочной и валидной выборки

In [16]:
features_train=pd.concat([features_train, features_valid])
target_train=pd.concat([target_train, target_valid])

In [17]:
model = RandomForestClassifier(random_state=12345, n_estimators=5, max_depth=13)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print('accuracy =','{:.4f}'.format(accuracy))

##  Проверьте модели на адекватность

Модель, которая предсказывает наиболее часто встречающийся класс

In [18]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)

Простейшая модель ошибается в 30%.Выбранная модель лучше, чем простейшая модель, проверка на адекватность пройдена.
